In [ ]:
%cd ..
import os
import time

In [ ]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
disable_cache()

In [ ]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"
if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
NUSCENES_RAW_DATA = "NUSCENES_RAW_DATA"
if NUSCENES_RAW_DATA in os.environ:
    RAW_DATA_DIR = os.environ[NUSCENES_RAW_DATA]
else:
    RAW_DATA_DIR = "/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini"

In [ ]:
import cv2
from os.path import exists
def construct_video(frames, size=(1600, 900), base=False, vid_name=None, vid_prefix=False):
    unique_frames = []
    for f in frames:
        if f not in unique_frames:
            unique_frames.append(f)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = "./outputs/fig13_"+vid_name if vid_name else f"./outputs/{'fig13_base' if base else 'fig13_optimized'}.mp4"
    display_video = cv2.VideoWriter(vid_name,fourcc, 1, size)
    for frame in unique_frames:
        if vid_prefix:
            img_path = os.path.join(RAW_DATA_DIR, 'sweeps/CAM_FRONT', frame)
            if not exists(img_path):
                img_path = os.path.join(RAW_DATA_DIR, 'samples/CAM_FRONT', frame)
        else:
            img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [ ]:
name = 'ScenicWorld'
world = empty_world(name=name)

obj1 = objects[0]
obj2 = objects[1]
cam = camera

# car_world = empty_world(name=name).filter(
#     (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck') | F.like(obj1.type, 'bus'))
# )
# world = world.filter(
#     (obj1.id != obj2.id) &
#     (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck')) &
#     (F.like(obj2.type, 'car') | F.like(obj2.type, 'truck')) &
#     F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego)), -15, 15) &
#     (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
#     (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2.0) &
#     (F.distance(cam.ego, obj2.trans@cam.time) < 50) &
#     (F.view_angle(obj2.trans@cam.time, cam.ego) < 70 / 2.0) &
#     F.contains_all('intersection', [obj1.trans, obj2.trans]@cam.time) &
#     F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 40, 135) &
#     F.angle_between(F.facing_relative(obj2.trans@cam.time, cam.ego), -135, -50) &
#     (F.min_distance(cam.ego, 'intersection') < 10) &
#     F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), 100, -100)
# )
world = world.filter(
    (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck')) &
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego)), -15, 15) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
    F.contains_all('intersection', [obj1.trans]@cam.time) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 135, 225) &
    (F.min_distance(cam.ego, 'intersection') < 10)
)


In [ ]:
preprocess(world, DATA_DIR, ['scene-0655-CAM_FRONT'], base=False)

In [ ]:
import numpy as np
investigation =[[2, 'car_exit_segment', ((577, 468), (632, 516))], [3, 'car_exit_segment', ((574, 468), (629, 517))], [4, None, None], [10, 'ego_exit_segment', None], [14, 'car_exit_segment', ((1, 433), (406, 629))], [15, 'car_exit_segment', ((505, 472), (582, 533))], [19, 'car_exit_segment', ((1, 429), (307, 653))], [22, 'car_exit_segment', ((471, 463), (560, 531))], [24, 'ego_exit_segment', None], [26, 'car_exit_segment', ((736, 474), (765, 497))], [27, 'car_exit_segment', ((745, 473), (796, 498))], [29, 'car_exit_segment', ((742, 473), (796, 498))], [30, 'car_exit_segment', ((735, 475), (769, 499))], [35, 'ego_exit_segment', None], [41, 'car_exit_segment', ((403, 465), (527, 552))], [46, 'car_exit_segment', ((321, 466), (481, 575))], [49, 'car_exit_segment', ((199, 462), (426, 597))], [53, 'car_exit_segment', ((44, 461), (360, 625))], [60, 'meet_up', ((2, 452), (264, 651))], [63, 'car_exit_segment', ((283, 474), (612, 663))], [116, 'car_exit_segment', ((674, 456), (735, 504))], [119, 'car_exit_segment', ((679, 454), (744, 502))], [120, None, None], [122, 'car_exit_segment', ((682, 450), (749, 502))], [123, 'car_exit_segment', ((683, 461), (750, 512))], [124, 'car_exit_segment', ((684, 468), (749, 516))], [125, 'car_exit_segment', ((686, 473), (750, 522))], [126, 'car_exit_segment', ((685, 472), (751, 522))], [127, 'car_exit_segment', ((686, 471), (751, 520))], [128, 'car_exit_segment', ((684, 462), (759, 514))], [129, 'car_exit_segment', ((682, 456), (759, 508))], [130, 'car_exit_segment', ((686, 451), (758, 506))], [131, 'car_exit_segment', ((687, 450), (761, 505))], [132, None, None], [133, None, None], [134, None, None], [135, None, None], [136, 'car_exit_segment', ((687, 470), (760, 527))], [137, 'car_exit_segment', ((685, 467), (763, 527))], [138, None, None], [139, 'car_exit_segment', ((3, 486), (259, 710))], [140, 'car_exit_segment', ((678, 474), (764, 539))], [141, 'car_exit_segment', ((673, 480), (759, 547))], [142, 'car_exit_segment', ((677, 487), (755, 550))], [144, 'car_exit_segment', ((670, 481), (755, 547))], [145, 'car_exit_segment', ((659, 466), (752, 537))], [146, None, None], [148, 'car_exit_segment', ((954, 475), (998, 499))], [149, None, None], [153, 'ego_exit_segment', None], [167, 'car_exit_segment', ((474, 439), (655, 560))], [169, 'car_exit_segment', ((456, 438), (652, 562))], [171, 'car_exit_segment', ((429, 436), (644, 566))], [172, 'car_exit_segment', ((393, 436), (630, 571))], [174, 'car_exit_segment', ((375, 435), (618, 575))], [176, 'car_exit_segment', ((353, 438), (610, 588))], [178, 'car_exit_segment', ((281, 431), (591, 606))], [179, 'car_exit_segment', ((229, 436), (566, 623))], [181, 'car_exit_segment', ((184, 436), (555, 630))], [182, 'ego_exit_segment', None], [184, 'car_exit_segment', ((70, 432), (530, 660))], [186, 'car_exit_segment', ((4, 421), (505, 685))], [187, 'car_exit_segment', ((1, 404), (444, 737))], [190, 'car_exit_segment', ((3, 396), (417, 759))], [195, 'ego_exit_segment', None], [212, 'car_exit_segment', ((728, 463), (785, 509))], [217, 'car_exit_segment', ((711, 452), (777, 508))], [222, 'car_exit_segment', ((682, 447), (766, 517))], [223, 'ego_exit_segment', None], [227, 'car_exit_segment', ((624, 453), (746, 550))], [228, None, None], [229, None, None], [230, None, None], [231, None, None], [232, None, None], [233, None, None], [234, None, None], [235, None, None], [236, None, None]]
target_190 = [[2, 761, 471, 797, 497, 'lanegroup',
           np.array([list(zip([681.3024786662291, 653.6754277216598, 733.4863798631264, 744.566365545458, 759.4397290738865, 810.2510673985199, 782.5420928392875, 681.3024786662291],
                            [514.2797277097048, 513.9338490065937, 497.1219793070591, 494.7131174212107, 494.88841836279886, 495.4692971363944, 515.4405856056001, 514.2797277097048]))]).reshape(-1,1,2)]]

target_207 = [[0, 728, 463, 785, 509, 'lanegroup',
           np.array([list(zip([632.5673278607485, 595.0064038929324, 723.9896110386784, 739.2077568337414, 756.3507025947408, 814.9641409917908, 770.6435112678597, 632.5673278607485],
                            [528.9128096702734, 528.3205624725372, 501.6543160656389, 498.422248120345, 498.66987355238507, 499.49252554956377, 530.8904272699301, 528.9128096702734]))]).reshape(-1,1,2)]]
skipping_info_0655=[[0, 'car_exit_segment', ((577, 468), (632, 516))], [2, 'car_exit_segment', ((574, 468), (629, 517))], [3, None, None], [4, 'ego_exit_segment', None], [10, 'car_exit_segment', ((1, 433), (406, 629))], [14, 'car_exit_segment', ((505, 472), (582, 533))], [15, 'car_exit_segment', ((1, 429), (307, 653))], [19, 'car_exit_segment', ((471, 463), (560, 531))], [22, 'ego_exit_segment', None], [24, 'car_exit_segment', ((736, 474), (765, 497))], [26, 'car_exit_segment', ((745, 473), (796, 498))], [27, 'car_exit_segment', ((742, 473), (796, 498))], [29, 'car_exit_segment', ((735, 475), (769, 499))], [30, 'ego_exit_segment', None], [35, 'car_exit_segment', ((403, 465), (527, 552))], [41, 'car_exit_segment', ((321, 466), (481, 575))], [46, 'car_exit_segment', ((199, 462), (426, 597))], [49, 'car_exit_segment', ((44, 461), (360, 625))], [53, 'meet_up', ((2, 452), (264, 651))], [60, 'car_exit_segment', ((283, 474), (612, 663))], [112, 'car_exit_segment', ((674, 456), (735, 504))], [116, 'car_exit_segment', ((679, 454), (744, 502))], [119, None, None], [120, 'car_exit_segment', ((682, 450), (749, 502))], [122, 'car_exit_segment', ((683, 461), (750, 512))], [123, 'car_exit_segment', ((684, 468), (749, 516))], [124, 'car_exit_segment', ((686, 473), (750, 522))], [125, 'car_exit_segment', ((685, 472), (751, 522))], [126, 'car_exit_segment', ((686, 471), (751, 520))], [127, 'car_exit_segment', ((684, 462), (759, 514))], [128, 'car_exit_segment', ((682, 456), (759, 508))], [129, 'car_exit_segment', ((686, 451), (758, 506))], [130, 'car_exit_segment', ((687, 450), (761, 505))], [131, None, None], [132, None, None], [133, None, None], [134, None, None], [135, 'car_exit_segment', ((687, 470), (760, 527))], [136, 'car_exit_segment', ((685, 467), (763, 527))], [137, None, None], [138, 'car_exit_segment', ((3, 486), (259, 710))], [139, 'car_exit_segment', ((678, 474), (764, 539))], [140, 'car_exit_segment', ((673, 480), (759, 547))], [141, 'car_exit_segment', ((677, 487), (755, 550))], [142, 'car_exit_segment', ((670, 481), (755, 547))], [144, 'car_exit_segment', ((659, 466), (752, 537))], [145, None, None], [146, 'car_exit_segment', ((954, 475), (998, 499))], [148, None, None], [149, 'ego_exit_segment', None], [166, 'car_exit_segment', ((474, 439), (655, 560))], [167, 'car_exit_segment', ((456, 438), (652, 562))], [169, 'car_exit_segment', ((429, 436), (644, 566))], [171, 'car_exit_segment', ((393, 436), (630, 571))], [172, 'car_exit_segment', ((375, 435), (618, 575))], [174, 'car_exit_segment', ((353, 438), (610, 588))], [176, 'car_exit_segment', ((281, 431), (591, 606))], [178, 'car_exit_segment', ((229, 436), (566, 623))], [179, 'car_exit_segment', ((184, 436), (555, 630))], [181, 'ego_exit_segment', None], [182, 'car_exit_segment', ((70, 432), (530, 660))], [184, 'car_exit_segment', ((4, 421), (505, 685))], [186, 'car_exit_segment', ((1, 404), (444, 737))], [187, 'car_exit_segment', ((3, 396), (417, 759))], [190, 'ego_exit_segment', None], [207, 'car_exit_segment', ((728, 463), (785, 509))], [212, 'car_exit_segment', ((711, 452), (777, 508))], [217, 'car_exit_segment', ((682, 447), (766, 517))], [222, 'ego_exit_segment', None], [223, 'car_exit_segment', ((624, 453), (746, 550))], [227, None, None], [228, None, None], [229, None, None], [230, None, None], [231, None, None], [232, None, None], [233, None, None], [234, None, None], [235, None, None]]

def investigation_skipping(video_filename, skipping_info):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = f"./outputs/{video_filename}_finalized.mp4"
    display_video = cv2.VideoWriter(vid_name, fourcc, 1, (1600, 900))
    video_path = os.path.join(DATA_DIR, "videos", video_filename+'.mp4')
    vidcap = cv2.VideoCapture(video_path)
    frame_idx = 0
    current_frame_num, current_action, current_target_bbox = skipping_info[0]
    current_skipping_info_idx = 0
    current_bbox_appearance = 0
    while(vidcap.isOpened()):
        success, image = vidcap.read()
        if not success:
            break
        next_skipping_info_idx = current_skipping_info_idx + 1
        if next_skipping_info_idx < len(skipping_info):
            next_frame_num, next_action, next_target_bbox = skipping_info[next_skipping_info_idx]
        if frame_idx in [192, 195, 198, 200, 212]:
            f = cv2.putText(image, current_action, (50, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.imwrite(f'./outputs/0655_frame_{frame_idx}.jpg', f)
        if frame_idx == 207:
            f = cv2.putText(image, current_action, (50, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            for e in target_207:
                did, x, y, x_w, y_h, t, pts1 = e
                cv2.putText(f, 'Car A', (x-20, y_h+10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.putText(f, t, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.rectangle(f, (x,y), (x_w, y_h), (0,255,0), 2)
                cv2.polylines(f, np.int32([pts1]), isClosed=True, color=(255,0,0), thickness = 2)
            cv2.imwrite('./outputs/0655_frame_207_skipping_investigation_new.jpg', f)
        if frame_idx == 190:
            f = cv2.putText(image, current_action, (50, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            for e in target_190:
                did, x, y, x_w, y_h, t, pts1 = e
                cv2.putText(f, 'Car A', (x-20, y_h+10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.putText(f, t, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.rectangle(f, (x,y), (x_w, y_h), (0,255,0), 2)
                cv2.polylines(f, np.int32([pts1]), isClosed=True, color=(255,0,0), thickness = 2)
            cv2.imwrite('./outputs/0655_frame_190_skipping_investigation_new.jpg', f)
        if frame_idx < next_frame_num:
            
            if current_action:
                image = cv2.putText(image, current_action, (50, 50),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            if current_target_bbox and current_bbox_appearance == 0:
                x, y = current_target_bbox[0]
                x_w, y_h = current_target_bbox[1]
                cv2.rectangle(image, (x, y), (x_w, y_h), (0,255,0), 4)
                current_bbox_appearance += 1
        
        display_video.write(image)
        frame_idx += 1
        if frame_idx == next_frame_num:
            current_skipping_info_idx = next_skipping_info_idx
            current_action = next_action
            current_target_bbox = next_target_bbox
            current_bbox_appearance = 0
    display_video.release()
investigation_skipping('boston-seaport-scene-0655-CAM_FRONT', skipping_info_0655)

In [ ]:
optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(1)
print(f'optimized query time: {time.time()-optimized_query_start}')

In [ ]:
optimized_results = [e[3] for e in id_time_camId_filename]
construct_video(optimized_results, vid_name="modified.mp4")

In [ ]:
preprocess(world, DATA_DIR, ['scene-0757-CAM_FRONT'])

In [ ]:
base_query_start = time.time()
id_time_camId_filename_base = world.get_id_time_camId_filename(2)
print(f'based query time: {time.time()-base_query_start}')

In [ ]:
id_time_camId_filename_base

In [ ]:
base_results = [e[4] for e in id_time_camId_filename_base]
construct_video(base_results, base=True)

In [ ]:
false_negative = [f for f in base_results if f not in optimized_results]
print(len(false_negative))
construct_video(false_negative, vid_name="false_negative.avi")

In [ ]:
false_positive = [f for f in optimized_results if f not in base_results]
print(len(false_positive))
construct_video(false_positive, vid_name="false_positive.avi")